In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
import scipy
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedKFold
from util import plot_history
import feature_builder

Cargo datasets

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Armo features

In [4]:
X_train = feature_builder.process_dataset(train_df)
y_train = train_df['target'].values

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 

In [6]:
X_train

,invalid_location_character_count,keyword_length,text_length,location_length,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,...,embedding_290,embedding_291,embedding_292,embedding_293,embedding_294,embedding_295,embedding_296,embedding_297,embedding_298,embedding_299
0,0,0,69,0,0.071930,0.072657,0.113349,0.144185,-0.081472,0.019648,...,-0.066101,0.075951,-0.080343,-0.041148,-0.024180,0.047680,-0.007228,-0.038648,0.018550,0.042705
1,0,0,38,0,0.085254,-0.011194,0.002753,0.136572,0.177734,-0.000635,...,-0.059473,-0.018774,-0.040625,0.167126,0.067529,0.010791,-0.139258,-0.042334,0.070020,0.054913
2,0,0,133,0,-0.001703,-0.007318,0.101742,0.020781,-0.058668,-0.047633,...,-0.109157,-0.050614,-0.083869,0.011757,0.087813,0.062526,-0.062730,-0.023630,0.021548,-0.032194
3,0,0,65,0,0.157750,-0.069493,0.024990,0.191389,0.032000,-0.082421,...,0.122890,-0.056501,-0.030562,0.029785,0.235526,0.047089,-0.197405,0.021240,0.010228,-0.048758
4,0,0,88,0,0.061076,0.021801,-0.027214,0.076986,-0.022819,-0.003373,...,-0.012956,-0.085065,-0.064406,0.055509,0.000789,0.008598,0.046381,-0.040745,0.040226,0.008915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,83,0,-0.026154,0.015155,-0.029889,0.046122,-0.014273,-0.066925,...,-0.133189,-0.001648,-0.060666,0.028127,-0.010269,-0.050395,-0.027944,-0.057841,-0.007520,-0.018351
7609,0,0,125,0,0.103575,0.039608,0.014465,0.057158,-0.026443,-0.073206,...,-0.022889,-0.029913,-0.056385,0.026202,0.009356,-0.012412,-0.064728,-0.011390,0.047949,-0.015955
7610,0,0,65,0,-0.154677,-0.025133,-0.008226,0.157661,-0.052558,0.045437,...,0.061900,0.121243,-0.032888,0.115614,-0.097493,-0.066135,-0.146349,-0.000258,0.022780,0.158312
7611,0,0,137,0,0.036671,0.045404,0.039435,-0.006173,-0.037481,0.003383,...,-0.009520,-0.035136,-0.067025,0.028713,-0.047666,-0.004495,0.018032,-0.015921,-0.000759,-0.014791


In [32]:
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

model = create_model()
history = model.fit(X_train, y_train, epochs=15, shuffle=True, validation_split=0.0)
plot_history(history)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 64)                19520     
_________________________________________________________________
dense_42 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 65        
Total params: 23,745
Trainable params: 23,745
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
238/238 [==============================] - 0s 987us/step - loss: 0.7055 - accuracy: 0.6199
Epoch 2/15
238/238 [==============================] - 0s 991us/step - loss: 0.5398 - accuracy: 0.7377
Epoch 3/15
238/238 [=======================

KeyError: 'val_accuracy'

In [29]:
test = feature_builder.process_dataset(test_df)
prediction = model.predict_classes(test)
pd.Series([x[0] for x in prediction]).value_counts()

0    2162
1    1101
dtype: int64

Calculemos el test para kaggle

In [34]:
ids = test_df['id']

final_test = feature_builder.process_dataset(test_df)
prediction = model.predict_classes(final_test)
final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

,target
id,
0,0
2,1
3,1
9,1
11,1
...,...
10861,1
10865,1
10868,1


In [35]:
final_df['target'].value_counts()

0    2005
1    1258
Name: target, dtype: int64

In [36]:
final_df.count()

target    3263
dtype: int64

In [37]:
final_df.to_csv('nn.csv')